<a href="https://colab.research.google.com/github/karnwatcharasupat/musi6001-music-gen/blob/main/music_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MUSI6001 Music Perception & Cognition: Cognitive Modeling and Music Generation

## Installation

In [1]:
!sudo apt install -y fluidsynth

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  fluid-soundfont-gm libfluidsynth2 libinstpatch-1.0-2 qsynth
  timgm6mb-soundfont
Suggested packages:
  fluid-soundfont-gs timidity jackd musescore
The following NEW packages will be installed:
  fluid-soundfont-gm fluidsynth libfluidsynth2 libinstpatch-1.0-2 qsynth
  timgm6mb-soundfont
0 upgraded, 6 newly installed, 0 to remove and 22 not upgraded.
Need to get 126 MB of archives.
After this operation, 157 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 fluid-soundfont-gm all 3.1-5.1 [119 MB]
Get:2 http://archive.ubuntu.com/ubuntu focal/universe amd64 libinstpatch-1.0-2 amd64 1.1.2-2build1 [238 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/universe amd64 timgm6mb-soundfont all 1.3-3 [5,420 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal/universe amd64 libfluidsynth2 amd64 2.1.1-2 [

In [2]:
!sudo apt install musescore3

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  desktop-file-utils fonts-freefont-ttf libpoppler-qt5-1 libportaudio2
  libportmidi0 libqt5help5 libqt5qml5 libqt5quick5 libqt5sql5-sqlite
  libqt5svg5 libqt5xmlpatterns5 musescore-general-soundfont-small
  musescore3-common qml-module-qtgraphicaleffects qml-module-qtquick-controls
  qml-module-qtquick-dialogs qml-module-qtquick-layouts
  qml-module-qtquick-privatewidgets qml-module-qtquick-window2
  qml-module-qtquick2
Suggested packages:
  qt5-qmltooling-plugins pulseaudio-utils
The following NEW packages will be installed:
  desktop-file-utils fonts-freefont-ttf libpoppler-qt5-1 libportaudio2
  libportmidi0 libqt5help5 libqt5qml5 libqt5quick5 libqt5sql5-sqlite
  libqt5svg5 libqt5xmlpatterns5 musescore-general-soundfont-small musescore3
  musescore3-common qml-module-qtgraphicaleffects qml-module-qtquick-controls
  qml-module-qtquick-di

In [3]:
!pip install --upgrade pyfluidsynth

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install pretty_midi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 27.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 KB 4.4 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592305 sha256=2672a3edee1b2b70e3de1c636587a2789b6dbd90ca58d31bbf2fffc1c527694d
  Stored in directory: /root/.cache/pip/wheels/96/4a/bd/ea2d2c62f684a0bcce6e6d277cb69e4463b5fd372c422c2481
Successfully built pretty_midi


In [5]:
!pip install librosa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
!pip install mingus

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.0/140.0 KB 4.6 MB/s eta 0:00:00


## Code

In [7]:
import numpy as np

from IPython.display import Audio
import soundfile as sf
import pretty_midi

from librosa import midi_to_note, note_to_midi
from mingus.core import keys, notes, chords, progressions
from mingus.core.intervals import from_shorthand

import os

In [8]:
# Adapted from Magenta's code

def display_audio(midi: pretty_midi.PrettyMIDI, out=None, fs=44100, limit_seconds=60):
  waveform = midi.fluidsynth(fs=fs)

  if out is not None:
    sf.write(out, waveform, fs)
  # Take a sample of the generated waveform to mitigate kernel resets
  waveform_short = waveform[:limit_seconds*fs]
  
  return Audio(waveform_short, rate=fs)

In [24]:
from itertools import chain
from collections import defaultdict

def respell_enharmonic(n):
  if "#" in n:
    return notes.int_to_note(notes.note_to_int(n), accidentals='b')
  elif "b" in n:
    return notes.int_to_note(notes.note_to_int(n))
  else:
    return n


def respell_chord(c):
  c0 = c
  n = c.lower().replace("7", "").replace("dim", "").replace("m", "").capitalize()
  f = c0.replace(n, "")
  n = respell_enharmonic(n)

  return n + f


class MusicActivation():
  def __init__(
      self,
      midi_min=36,
      midi_max=96,
      note_decay=0.9,
      chord_decay=0.99,
      key_decay=0.999,
      curr_key_mult=100,
  ):
    self.indices = np.arange(midi_max-midi_min+1)
    self.midis = np.arange(midi_min, midi_max+1)
    self.notes = np.array(
        [midi_to_note(m, octave=False, unicode=False) for m in self.midis]
    )
    self.note_activations = np.zeros_like(self.notes, dtype=float)

    self.keys = np.array(keys.major_keys + keys.minor_keys)
    self.key_activations = np.zeros_like(self.keys, dtype=float)

    self.chords = np.array(list(chain(*[
        [f"{k}M", f"{k}m", f"{k}dim", f"{k}M7", f"{k}m7"] for k in [notes.int_to_note(i) for i in range(12)]
    ])))
    self.chord_activations = np.zeros_like(self.chords, dtype=float)

    self.notes_in_chord = {
        c: list(set([respell_enharmonic(n) for n in chords.from_shorthand(c)])) for c in self.chords
    }

    self.chords_in_note = defaultdict(set)

    for c, ns in self.notes_in_chord.items():
      for n in ns:
        self.chords_in_note[respell_enharmonic(n)].add(c)

    self.chords_in_key = defaultdict(set)
    self.keys_in_chord = defaultdict(set)

    for k in self.keys:

      chords_of_key = [
            chords.determine_triad(c, shorthand=True, no_inversions=True)[0] for c in [
                chords.tonic(k), 
                chords.supertonic(k), 
                chords.mediant(k), 
                chords.subdominant(k), 
                chords.dominant(k), 
                chords.submediant(k), 
                chords.subtonic(k),
            ]
        ] + [
            chords.determine_seventh(c, shorthand=True)[0] for c in [
                chords.tonic7(k), 
                chords.supertonic7(k), 
                chords.mediant7(k), 
                chords.subdominant7(k), 
                chords.dominant7(k), 
                chords.submediant7(k), 
                chords.subtonic7(k),
            ]
        ] 
      self.chords_in_key[k] = chords_of_key

      for c in chords_of_key:
        self.keys_in_chord[c].add(k)


    self.chords_in_note = {k: list(v) for k, v in self.chords_in_note.items()}
    self.chords_in_key = {k: list(v) for k, v in self.chords_in_key.items()}
    self.keys_in_chord = {k: list(v) for k, v in self.keys_in_chord.items()}

    self.curr_key = None
    self.curr_chord = None

    self.curr_key_mult = curr_key_mult
    # self.curr_chord_mult = curr_chord_mult

    self.note_decay = note_decay
    self.chord_decay = chord_decay
    self.key_decay = key_decay
  
  def activate_chords_from_note(self, note):
    # print(note)
    if note not in self.chords_in_note:
      note = respell_enharmonic(note)
    
    chord_act = np.zeros_like(self.chord_activations)

    activated_chords = self.chords_in_note[note]
    for i, c in enumerate(self.chords):
      if c in activated_chords:
        chord_act[i] += 1

    self.chord_activations += chord_act

    return chord_act


  def activate_keys_from_chords(self, chord_act):
    key_act = np.zeros_like(self.key_activations)
    
    for c in self.chords[chord_act > 0]:
      
      if c not in self.keys_in_chord:
        c = respell_chord(c)
      activated_keys = self.keys_in_chord[c]
      for i, k in enumerate(self.keys):
        if k in activated_keys:
          key_act[i] += 1

    self.key_activations += key_act
    
    return key_act

  
  def activate_chords_from_keys(self, key_act):
    chord_act = np.zeros_like(self.chord_activations)

    for k in self.keys[key_act > 0]:
      if k == self.curr_key:
        act = self.curr_key_mult
      else:
        act = 1

      activated_chords = self.chords_in_key[k]

      for i, c in enumerate(self.chords):
        if c in activated_chords:
          chord_act[i] += act

    self.chord_activations += chord_act

    return chord_act

  def activate_notes_from_chords(self, chord_act):
    note_act = np.zeros_like(self.note_activations)

    for c in self.chords[chord_act > 0]:
      activated_notes = self.notes_in_chord[c]
      # print(activated_notes)

      for i, n in enumerate(self.notes):
        if n in activated_notes or respell_enharmonic(n) in activated_notes:
          note_act[i] += 1

    self.note_activations += note_act

    return note_act
   

  def activate(
      self,
      index,
      note
  ):
    
    self.key_activations *= self.key_decay
    self.chord_activations *= self.chord_decay 
    self.note_activations *= self.note_decay
    
    chord_act = self.activate_chords_from_note(note)
    # print(self.chord_activations)
    key_act = self.activate_keys_from_chords(chord_act)
    # print(self.key_activations)
    chord_act = self.activate_chords_from_keys(key_act)
    # print(self.chord_activations)
    _ = self.activate_notes_from_chords(chord_act)
    # print(self.note_activations)


    # proximity rule
    prox = np.power(np.maximum(np.abs(self.indices - index), 4.0), -2.0)
    prox = (prox - np.min(prox))/np.ptp(prox) * np.std(self.note_activations)
    # print(prox)
    self.note_activations += prox
    # plt.plot(self.note_activations)
    # plt.show()

  def next_note(self, p=None):
    if p is None:
      p = self.note_activations/np.sum(self.note_activations)
    else:
      p = p/np.sum(p)

    i = np.random.choice(self.indices, p=p)
    note = self.notes[i]

    return i, note

  def current_chord(self):
    p = self.chord_activations/np.sum(self.chord_activations)
    c = np.random.choice(self.chords, p=p)

    self.curr_chord = c

    return c

  def current_key(self):
    k = self.keys[np.argmax(self.key_activations)]

    self.curr_key = k

    return k

  def generate(self, max_notes=128, discard_first_n=0):

    note_out = [None for _ in range(max_notes)]
    chord_out = [None for _ in range(max_notes)]

    prev_key = None

    for t in range(discard_first_n):
      i, note = self.next_note(p=np.ones_like(self.midis) if t == 0 else None)
      self.activate(i, note)

    for t in range(max_notes):
      i, note = self.next_note(p=np.ones_like(self.midis) if (t == 0 and discard_first_n == 0) else None)
      self.activate(i, note)
      curr_key = self.current_key()
      chord_out[t] = self.current_chord()
      note_out[t] = self.midis[i]      

      if t > max_notes/2:
        prog = progressions.determine([chords.from_shorthand(chord_out[t-1]), chords.from_shorthand(chord_out[t])], 
             curr_key, 
             shorthand=True)
        prog = [p[0].replace("7", "") for p in prog]
        # print(prog)

        if prog == ["V", "I"]:
          print("PERFECT CADENCE")
          break
        

      if prev_key is not None and prev_key != curr_key:
        print(f"KEY CHANGE @ {t}: {prev_key} --> {curr_key}")

      prev_key = curr_key

    note_out = note_out[:t]
    chord_out = chord_out[:t]

    return note_out, chord_out

In [25]:
def generate(filename, discard_first_n=32):

  ma = MusicActivation(midi_min=note_to_midi('F4'), midi_max=note_to_midi('F6'))
  seq = ma.generate(max_notes=64, discard_first_n=discard_first_n)

  bpm = 100
  pm = pretty_midi.PrettyMIDI(resolution=220, initial_tempo=bpm)
  cello_program = pretty_midi.instrument_name_to_program('Piccolo')
  cello = pretty_midi.Instrument(program=cello_program)

  piano_program = pretty_midi.instrument_name_to_program('Vibraphone')
  piano = pretty_midi.Instrument(program=piano_program)

  qlen = 60.0/bpm

  start = 0

  beats = 0
  prevBeats = -1
  chorded = False

  n_notes = len(seq[0])

  for t, (m, c) in enumerate(zip(*seq)):


    if t == n_notes - 1:
      dur = 2
    elif t == n_notes - 2:
      dur = 0.5
    elif t == n_notes - 3:
      dur = 1.5
    elif t == n_notes - 4:
      dur = np.ceil(start) - start
      if dur == 0:
        dur = 1
    else:
      if np.round(beats) - beats == 0.5:
        dur = np.random.choice([0.25, 0.5, 0.75, 1], p=[0.5, 0.375, 0, 0.125])
      elif np.ceil(beats) - beats == 0.25:
        dur = 0.25
      elif np.ceil(beats) - beats == 0.75:
        dur = np.random.choice([0.25, 0.5, 0.75, 1], p=[0.25, 0.375, 0.375, 0])
      else:
        dur = np.random.choice([0.25, 0.5, 0.75, 1], p=[0.25, 0.375, 0.25, 0.125])

    note = pretty_midi.Note(velocity=64, pitch=m, start=start, end=start+dur*qlen)
    cello.notes.append(note)

    if not chorded:
      for i, n in enumerate(chords.from_shorthand(c)):
        piano.notes.append(
            pretty_midi.Note(
                velocity=48, 
                pitch=pretty_midi.note_name_to_number(notes.reduce_accidentals(n) + "3"), 
                start=start+i*0.25*qlen, 
                end=start+1.5*qlen
            )
        )
      chorded = True
    
    start += dur*qlen
    prevBeats = beats
    beats += dur

    if np.floor(prevBeats/2) < np.floor(beats/2):
      chorded = False

  pm.instruments.append(cello)
  pm.instruments.append(piano)

  os.makedirs(os.path.join("/content", "musgen", filename), exist_ok=True)

  pm.write(os.path.join("/content", "musgen", filename, f"{filename}.mid"))

  display_audio(pm, out=os.path.join("/content", "musgen", filename, f"{filename}.wav"))

In [27]:
from tqdm.notebook import tqdm

np.random.seed(42)

for i in tqdm(range(50)):
  print(f"Generating {i}th piece")
  generate(f"{i:02d}")

  0%|          | 0/50 [00:00<?, ?it/s]

Generating 0th piece
KEY CHANGE @ 13: A --> D
KEY CHANGE @ 14: D --> A
KEY CHANGE @ 38: A --> F#
PERFECT CADENCE
Generating 1th piece
KEY CHANGE @ 13: A --> E
KEY CHANGE @ 14: E --> A
Generating 2th piece
KEY CHANGE @ 2: E --> A
KEY CHANGE @ 4: A --> D
KEY CHANGE @ 6: D --> E
KEY CHANGE @ 7: E --> D
KEY CHANGE @ 14: D --> A
KEY CHANGE @ 16: A --> D
KEY CHANGE @ 32: D --> E
KEY CHANGE @ 56: E --> D
Generating 3th piece
KEY CHANGE @ 4: D --> G
PERFECT CADENCE
Generating 4th piece
KEY CHANGE @ 1: F --> Bb
KEY CHANGE @ 4: Bb --> F
KEY CHANGE @ 6: F --> B
KEY CHANGE @ 8: B --> C
KEY CHANGE @ 11: C --> F
KEY CHANGE @ 15: F --> C
KEY CHANGE @ 18: C --> F
KEY CHANGE @ 21: F --> C
KEY CHANGE @ 28: C --> B
KEY CHANGE @ 32: B --> E
KEY CHANGE @ 34: E --> B
KEY CHANGE @ 48: B --> E
KEY CHANGE @ 49: E --> B
Generating 5th piece
Generating 6th piece
Generating 7th piece
KEY CHANGE @ 8: E --> A
KEY CHANGE @ 9: A --> E
KEY CHANGE @ 19: E --> A
KEY CHANGE @ 57: A --> E
KEY CHANGE @ 58: E --> A
KEY CHAN

In [28]:
import shutil
shutil.make_archive("/content/musgen", 'zip', "/content/musgen")

'/content/musgen.zip'